In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import tensorflow as tf
from optuna.integration import TFKerasPruningCallback
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, r2_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping  # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint  # type: ignore
from tensorflow.keras.layers import LSTM, Dense, Dropout  # type: ignore
from tensorflow.keras.models import Sequential, load_model  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
from joblib import Parallel, delayed, parallel_backend


In [ ]:
# 2. Charger les données
# Charger les données prétraitées

df = pd.read_csv("../datasets/Alcohol_Sales.csv", parse_dates=True, index_col=0)
df.dropna(inplace=True)

# S'assurer que l'index est une colonne de type datetime
df.index = pd.to_datetime(df.index)

# if len(df.columns) == 1 : df['target'] = df[df.columns[-1]]



# Identifier les colonnes cibles (toutes sauf la date)
target_column = df.columns[-1]

feature_columns = [col for col in df.columns if col != target_column] if len(df.columns) > 1 else [target_column]




In [ ]:
# 3. Normalisation des données
# LSTM fonctionne mieux avec des données normalisées entre 0 et 1 :
scalers = {}
scaled_data = {}

scalers = {}

for column in df.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data[column] = scaler.fit_transform(df[[column]])
    scaled_data[column] = scaled_data[column].flatten()
    scalers[column] = scaler

scaled_data = pd.DataFrame(scaled_data, index=df.index)
scaled_data.reset_index(inplace=True)
train_size = int(len(scaled_data) * 0.8)
X_train = scaled_data.iloc[:train_size][feature_columns]
y_train = scaled_data.iloc[:train_size][target_column]
X_test = scaled_data.iloc[train_size:][feature_columns]
y_test = scaled_data.iloc[train_size:][target_column]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# scaler_X = MinMaxScaler(feature_range=(0, 1))
# scaler_y = MinMaxScaler(feature_range=(0, 1))

# # Fit scaler on training data only
# X_train_scaled = scaler_X.fit_transform(X_train)
# y_train_scaled = scaler_X.transform(pd.DataFrame(y_train))
# # y_train_scaled = scaler_y.fit_transform(pd.DataFrame(y_train))

# # Transform test data with the same scaler
# X_test_scaled = scaler_X.transform(X_test)
# y_test_scaled = scaler_X.transform(pd.DataFrame(y_test))

y_train

In [ ]:
X_train

In [ ]:
# # 4. Transformer les données en séquences pour LSTM
# def create_sequences(data, seq_length=12):
#     sequences = []
#     for i in range(len(data) - seq_length):
#         seq = data[i:i+seq_length]
#         sequences.append(seq)
#     return np.array(sequences)

# #seq_length: Nombre de pas de temps utilisés pour la prédiction
# detected_freq = pd.infer_freq(df.index)
# print("***",detected_freq,"***")
# if detected_freq in ["D", "B"]:  # Quotidienne ou Business Days
#     seq_length = 10
# elif detected_freq in ["MS", "M"]:# "justify your choices"
#     seq_length = 12
# else:
#     seq_length = 1
# print(seq_length)
# # Dictionnaire pour stocker les séquences

# X_train_seq = create_sequences(X_train, seq_length=seq_length)
# y_train_seq = create_sequences(y_train, seq_length=seq_length)
# X_test_seq = create_sequences(X_test, seq_length=seq_length)
# y_test_seq = create_sequences(y_test, seq_length=seq_length)


In [ ]:
def create_sequences(X, y, seq_length=12):
    Xs, ys = [], []
    for i in range(len(X) - seq_length):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i + seq_length])  # Predict one step ahead
    return np.array(Xs), np.array(ys)

# Detect frequency to set sequence length dynamically
detected_freq = pd.infer_freq(df.index)
print("***", detected_freq, "***")

if detected_freq in ["D", "B"]:  # Daily or Business Days
    seq_length = 10
elif detected_freq in ["MS", "M"]:  # Monthly Start or Monthly
    seq_length = 12
else:
    seq_length = 1

print("Sequence length:", seq_length)

# Prepare the data (use .values to get numpy arrays)
X_train_seq, y_train_seq = create_sequences(X_train.values, y_train.values, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test.values, y_test.values, seq_length)


In [ ]:
y_train

In [ ]:
def objective(trial):
    model = Sequential()

    # Hyperparameters
    n_layers = trial.suggest_int('n_layers', 1, 3)
    units = trial.suggest_int('units', 32, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    for i in range(n_layers):
        return_seq = (i < n_layers - 1)
        if i == 0:
            model.add(LSTM(units,activation='tanh', return_sequences=return_seq, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
        else:
            model.add(LSTM(units,activation='tanh', return_sequences=return_seq))
        model.add(Dropout(dropout_rate))

    model.add(Dense(units=1, activation='linear'))  # Regression

    model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5, restore_best_weights=True)
    pruning_callback = TFKerasPruningCallback(trial, "val_loss")

    # Save best model of this trial
    model_path = f"BestModel.keras"
    checkpoint = ModelCheckpoint(model_path, monitor="val_loss", save_best_only=True)

    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_test_seq, y_test_seq),
        epochs=100,
        batch_size=16,
        callbacks=[early_stopping, pruning_callback, checkpoint],
        verbose=0
    )

    return min(history.history["val_loss"])

In [ ]:
pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(direction='minimize', pruner=pruner)
study.optimize(objective, n_trials=50)

print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ", study.best_trial.params)


In [ ]:
study.best_params

In [ ]:
# 6. Définir l'architecture du modèle LSTM
def build_lstm_model():
    model = Sequential([
        LSTM(200, activation='relu', return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        LSTM(200, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    model.fit(X_train_seq, y_train_seq, epochs=200, batch_size=16, verbose=1)

    return model

def build_lstm_model_2():
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_seq, y_train_seq, epochs=200, batch_size=16, verbose=1)
    return model



def build_optuna_lstm_model():
    pruner = optuna.pruners.MedianPruner(n_warmup_steps=5)
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=50)
    best_params = study.best_params

    model = Sequential()
    for i in range(best_params["n_layers"]):
        return_seq = i < best_params["n_layers"] - 1
        if i == 0:
            model.add(
                LSTM(
                    best_params["units"],
                    activation="relu",
                    return_sequences=return_seq,
                    input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]),
                )
            )
        else:
            model.add(
                LSTM(
                    best_params["units"], activation="relu", return_sequences=return_seq
                )
            )
        model.add(Dropout(best_params["dropout_rate"]))

    model.add(Dense(units=1, activation="linear"))

    model.compile(
        optimizer=Adam(best_params["learning_rate"]), loss="mse", metrics=["mse"]
    )

    model.fit(
        X_train_seq,
        y_train_seq,
        validation_data=(X_test_seq, y_test_seq),
        epochs=100,
        batch_size=16,
        # callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5, restore_best_weights=True)],
        verbose=1,
    )

    return model


In [ ]:
with parallel_backend("loky"):    
    models = Parallel(n_jobs=3)(
        delayed(func)()
        for func in [
            build_lstm_model_2,
            build_lstm_model,
            # build_optuna_lstm_model
        ]
    )

In [ ]:
models

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model = build_lstm_model()
# history = model.fit(
#     X_train_seq, y_train_seq,
#     epochs=100,
#     batch_size=32,
#     validation_split=0.2,
#     callbacks=[early_stopping],
#     verbose=1
# )

best_params = study.best_params

model = Sequential()
for i in range(best_params['n_layers']):
    return_seq = (i < best_params['n_layers'] - 1)
    if i == 0:
        model.add(LSTM(best_params['units'], activation='relu', return_sequences=return_seq, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    else:
        model.add(LSTM(best_params['units'], activation='relu', return_sequences=return_seq))
    model.add(Dropout(best_params['dropout_rate']))

model.add(Dense(units=1, activation='linear'))

model.compile(optimizer=Adam(best_params['learning_rate']), loss='mse', metrics=['mse'])

model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=100,
    batch_size=16,
    # callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5, restore_best_weights=True)],
    verbose=1
)


train_loss = model.evaluate(X_train_seq, y_train_seq, verbose=0)
test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'Train Loss: {train_loss}')
print(f'Test Loss: {test_loss}')

# Make predictions
predictions = model.predict(X_test_seq)
predictions

In [ ]:
y_test_rescaled = scalers[target_column].inverse_transform(y_test.values.reshape(-1, 1))  # For actual values
y_pred_rescaled = scalers[target_column].inverse_transform(predictions)
y_train_rescaled = scalers[target_column].inverse_transform(y_train.values.reshape(-1, 1))

plt.figure(figsize=(10, 6))
plt.plot(y_train.index, y_train_rescaled, label="Train")
plt.plot(y_test.index, y_test_rescaled, label="Test", color="orange")
plt.plot(y_test[seq_length:].index, y_pred_rescaled.flatten(), label="Forecast", linestyle="dashed", color="red")
plt.title('Actual vs Predicted Values')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
mse = mean_squared_error(y_test_rescaled[seq_length:], y_pred_rescaled)
mae = mean_absolute_error(y_test_rescaled[seq_length:],y_pred_rescaled )
r2 = r2_score(y_test_rescaled[seq_length:],y_pred_rescaled)
mape=mean_absolute_percentage_error(y_test_rescaled[seq_length:], y_pred_rescaled)

print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

In [ ]:
y_pred_rescaled